In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal
from keras import regularizers

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def vgg16(act):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('softmax'))

    return model

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = vgg16('tanh')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 56s 107ms/step - loss: 8.2756 - accuracy: 0.0110 - top3_acc: 0.0316 - top_k_categorical_accuracy: 0.0517 - val_loss: 6.8926 - val_accuracy: 0.0087 - val_top3_acc: 0.0304 - val_top_k_categorical_accuracy: 0.0489
Epoch 2/50
313/313 [==============================] - 30s 96ms/step - loss: 7.9894 - accuracy: 0.0099 - top3_acc: 0.0296 - top_k_categorical_accuracy: 0.0485 - val_loss: 6.8761 - val_accuracy: 0.0087 - val_top3_acc: 0.0277 - val_top_k_categorical_accuracy: 0.0460
Epoch 3/50
313/313 [==============================] - 30s 95ms/step - loss: 7.7662 - accuracy: 0.0097 - top3_acc: 0.0298 - top_k_categorical_accuracy: 0.0516 - val_loss: 6.8630 - val_accuracy: 0.0097 - val_top3_acc: 0.0272 - val_top_k_categorical_accuracy: 0.0482
Epoch 4/50
313/313 [==============================] - 31s 99ms/step - loss: 7.5764 - accuracy: 0.0101 - top3_acc: 0.0309 - top_k_categorical_accuracy: 0.0498 - val_loss: 6.8317 - val_accuracy: 0.0085 - val_t

In [8]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [8.200616836547852, 7.93467903137207, 7.718003273010254, 7.539983749389648, 7.405293941497803, 7.321521759033203, 7.239022731781006, 7.167513370513916, 7.101108551025391, 7.045865058898926, 7.0007548332214355, 6.962065696716309, 6.913015842437744, 6.87953519821167, 6.855572700500488, 6.816949367523193, 6.787607669830322, 6.762834548950195, 6.7399163246154785, 6.71868371963501, 6.694786071777344, 6.669783592224121, 6.65110445022583, 6.633155345916748, 6.614596843719482, 6.595661640167236, 6.578643798828125, 6.558106422424316, 6.543043613433838, 6.52556037902832, 6.508054733276367, 6.491768836975098, 6.482688903808594, 6.466277122497559, 6.451168060302734, 6.437562942504883, 6.419929504394531, 6.4073944091796875, 6.3931169509887695, 6.381327152252197, 6.371471881866455, 6.359565734863281, 6.343942642211914, 6.331972599029541, 6.320064067840576, 6.307910442352295, 6.293668270111084, 6.2826151847839355, 6.273346424102783, 6.260302543640137], 'accuracy': [0.010350000113248825, 0.01